Будем работать с [логами рекомендательной системы фильмов](https://grouplens.org/datasets/movielens/). Основой данных будут две таблицы. Первая — это данные о выставленных оценках фильмов (ratings.csv):

In [1]:
import pandas as pd
ratings = pd.read_csv('../data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


- userId — идентификатор пользователя, который поставил фильму оценку
- movieId — идентификатор фильма
- rating — выставленная оценка
- timestamp — время (в формате unix time), когда была выставлена оценка

Вторая таблица — расшифровка идентификаторов фильмов (movies.csv):

In [ ]:
movies = pd.read_csv('movies.csv')
movies.head()

- movieId — идентификатор фильма
- title — название фильма
- genres — список жанров, к которым относится фильм

Тема объединения датафреймов в Pandas состоит из двух частей. Первая — это аналог SQL JOIN и более простое склеивание с помощью совмещения одинаковых столбцов датафреймов.

Вторая часть связана с перебором файлов в папках. Довольно часто данные будут попадать к вам не единой таблицей, а набором файлов. Работать с файлами вручную тяжело, поэтому мы рассмотрим методы автоматизации этого процесса с помощью библиотеки os.

- Наглядный [пример](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) различных режимов склейки таблиц по строкам или столбцам (метод concat). Пригодится, чтобы быстро вспомнить, как изменять типы объединения таблиц.
- Документация [метода merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) — пригодится, если забылись названия основных параметров (по каким столбцам объединяем таблицы и каким способом).
- [Объяснение](http://www.skillz.ru/dev/php/article-Obyasnenie_SQL_obedinenii_JOIN_INNER_OUTER.html) типов объединений таблиц — если возникнут трудности с выбором типа объединения (в pandas.merge параметр how).

У датафреймов ratings и movies есть общий столбец movieId. Значит, мы можем объединить эти датафреймы в одну таблицу. Используем метод merge:

In [ ]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head() 

Схематично метод merge можно описать так: joined = left_df.merge(right_df, on='', how='').

Давайте разберем подробнее параметры метода: 

- left_df / right_df — датафреймы, которые мы объединяем. К "правому" датафрейму присоединяем "левый" (в нашем примере "левый" датафрейм — ratings, "правый" — movies). 
- how — параметр объединения записей. Он может иметь четыре значения: left, right, inner и outer. При значении left берем все записи (movieId) из "левого" датафрейма (ratings) и ищем их соответствия в "правом" (movies). В итоговом датафрейме останутся только те значения, которым были найдены соответствия, то есть только значения из ratings. Аналогично при параметре right остаются только значения из "правого" датафрейма. Если совпадений между таблицами нет, то ставим нулевое значение. Значение inner оставляет только те записи (movieId), которые есть в обоих датафреймах, outer объединяет все варианты movieId в обоих датафреймах. 
- on определяет, по какому столбцу происходит объединение. Для объединения по нескольким столбцам используйте on = ['col1', 'col2'] или left_on и right_on.

После объединения датафреймов лучше проверять, что не возникло дубликатов. О возможных проблемах метода merge мы поговорим в следующем блоке. Сейчас убедимся в том, что число строк объединенного датафрейма совпадает с исходным:

После объединения датафреймов лучше проверять, что не возникло дубликатов. О возможных проблемах метода merge мы поговорим в следующем блоке. Сейчас убедимся в том, что число строк объединенного датафрейма совпадает с исходным:

In [ ]:
len(ratings) == len(joined)

Получаем значение True — значит, число строк совпадает.

Объединение датафреймов с помощью метода merge имеет особенности, аналогичные SQL JOIN. Если точнее, есть ситуации, которые приводят к дублированию строк в конечном результате. Разберем эти ситуации более подробно на примере небольших таблиц: ratings_example.txt и movies_example.txt.

В этих таблицах всего несколько строк, что позволит наглядно проверить, где дублируются результаты. Скачайте эти файлы себе и импортируйте в датафреймы ratings и movies. Обратите внимание, что разделителем столбцов в этих файлах является табуляция (\t):

In [ ]:
import pandas as pd
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

In [ ]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

**Дубликаты строк**

Итак, в датафрейме movies есть две строки с одним movieId. То есть теперь для таблицы ratings нет однозначного соответствия, с какой строкой она может объединиться с таблицей movies. В итоге строка с movieId = 31 будет дублирована:

In [ ]:
ratings.merge(movies, how = 'left', on = 'movieId')

**Удаляем дубликаты**

Если вы хотите избежать подобной ситуации, необходимо удалить дубликаты из таблицы movies. Для этого подходит метод drop_duplicates. В параметре subset указываем один или несколько столбцов, по комбинации которых хотим удалить дубликаты.

С помощью параметра keep указываем, какой из встречающихся дубликатов оставить (например, первый или последний). Параметр inplace указывает, что изменения нужно сохранить в датафрейме, к которому применяется метод (в нашем случае — в датафрейме movies):

In [ ]:
movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
movies.head()

Теперь объединение таблиц будет корректным:

In [ ]:
ratings.merge(movies, how = 'left', on = 'movieId')

### Задание

В этой серии заданий мы разберемся с данными новых поступлений интернет-магазина. В словаре items_dict (который мы переведем в датафрейм) содержится информация о наличии товара на складе:

In [ ]:
items_dict = {

    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 

    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],

    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]

}

А в словаре purchase_log — данные о покупках товаров:

In [ ]:
purchase_log = {

    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],

    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 

    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]

}

- item_id — идентификатор модели (по этому столбцу будем объединять датафреймы)
- vendor — производитель модели
- stock_count — имеющееся на складе количество данных моделей (в штуках)
- purchase_id — идентификатор покупки
- price — стоимость модели в покупке

Переведем сначала эти словари в датафреймы для удобства работы:

In [ ]:
items_df = pd.DataFrame(items_dict)
purchase_df = pd.DataFrame(purchase_log)

### Задание 1

Объедините получившиеся датафреймы по столбцу item_id с типом outer.

Определите, модель с каким item_id есть в статистике продаж purchase_df, но не учтена на складе (подсказка: подумайте, какой датафрейм должен быть "левым", а какой "правым", чтобы получить необходимые данные). Введите ответ в виде целого числа.

### Задание 2

Решите обратную задачу: модель с каким item_id есть на складе, но не имела ни одной продажи? Введите ответ в виде целого числа.

### Задание 3

Сформируйте датафрейм merged, в котором в результате объединения purchase_df и items_df останутся модели, которые учтены на складе и имели продажи. Сколько всего таких моделей?

### Задание 4

Посчитайте объем выручки для каждой модели, которую можно получить, распродав все остатки на складе. Модель с каким item_id имеет максимальное значение выручки после распродажи остатков? Ответ дайте в виде целого числа.

Примечание: перемножение столбцов датафрейма можно производить разными способами, но самый простой - перемножение "в лоб" вида df['col1'] = df['col2'] * df['col3']. Для присоединения новых данных к датафрейму тоже можно использовать различные методы, включая функцию .append(), которая позволяет присоединять к датафрейму другой датафрейм, серии или словари.

### Задание 5

Посчитайте итоговую выручку из прошлого задания по всем моделям. Ответ дайте в виде целого числа.